# Research Assistant Exercise

In this exercise, we'll create a research assistant that:
1. Breaks down complex topics into subtopics
2. Synthesizes information
3. Creates structured summaries

Focus on crafting research and analysis prompts!

In [ ]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

sys.path.append(grandparent_dir)

### 1. Setup 

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.planners import FunctionCallingStepwisePlanner, FunctionCallingStepwisePlannerOptions

kernel = Kernel()

# Chat service
service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)
kernel.add_service(chat_service)

# Embedding service
embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

# Memory setup
memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))

# Planner setup
planner = FunctionCallingStepwisePlanner(
    service_id=service_id,
    options=FunctionCallingStepwisePlannerOptions(
        max_iterations=10,
        max_tokens=4000,
    )
)

### 2. Define Base Prompts (Experiment with these!)

These prompts work together to analyze topics. Try modifying them to change how research is conducted!

In [ ]:
# Prompt 1: Break down research topic
# Try changing the structure or adding different analysis angles!
breakdown_prompt = """
Research Topic: {{$topic}}
Depth Level: {{$depth}}

Break down this topic into key areas for research.
Create a structured exploration plan.

Format the breakdown as:
📚 Main Topic: [topic name]

🔍 Key Areas:
1. [subtopic]
   - [aspects to explore]
   - [key questions]

2. [subtopic]
   - [aspects to explore]
   - [key questions]

🎯 Research Goals:
- [what to understand]
- [what to discover]

⚡ Priority Areas:
- [most important aspects]
"""

# Prompt 2: Analyze information
# Try adding different analysis frameworks or perspectives!
analysis_prompt = """
Content: {{$content}}
Focus Area: {{$focus}}

Analyze this information deeply and critically.
Look for patterns, connections, and implications.

Format the analysis as:
📊 Key Findings:
- [main points]

🔄 Patterns & Trends:
- [identified patterns]
- [emerging trends]

🤔 Critical Analysis:
- Strengths: [what's well-supported]
- Gaps: [what's missing]
- Questions: [what needs more research]

💡 Insights:
- [key takeaways]
- [implications]
"""

# Prompt 3: Synthesize findings
# Try different synthesis approaches or output formats!
synthesis_prompt = """
Research Elements: {{$elements}}
Audience: {{$audience}}

Synthesize the research findings into a coherent narrative.
Make complex ideas accessible while maintaining depth.

Format the synthesis as:
📝 Executive Summary:
- [key points for quick understanding]

🔍 Detailed Findings:
1. [major finding]
   - Evidence: [supporting information]
   - Impact: [implications]

🔄 Interconnections:
- [how findings relate]
- [broader implications]

📈 Future Directions:
- [next steps]
- [areas for deeper research]
"""

# Create semantic functions
create_breakdown = kernel.add_function(
    prompt=breakdown_prompt,
    function_name="BreakdownTopic",
    plugin_name="ResearchAssistant",
)

analyze_info = kernel.add_function(
    prompt=analysis_prompt,
    function_name="AnalyzeInfo",
    plugin_name="ResearchAssistant",
)

synthesize_findings = kernel.add_function(
    prompt=synthesis_prompt,
    function_name="SynthesizeFindings",
    plugin_name="ResearchAssistant",
)

### 3. Example Usage

Try these examples, then modify the prompts above to see how it changes the results!

In [ ]:
# Example 1: Break down a research topic
topic_details = {
    "topic": "The impact of artificial intelligence on creative industries",
    "depth": "intermediate"
}

breakdown = await kernel.invoke(
    create_breakdown,
    **topic_details
)
print("Topic Breakdown:\n", breakdown)

# Example 2: Analyze a specific aspect
sample_content = """
AI tools are increasingly being used in art, music, and writing. While they enhance
productivity and enable new forms of expression, concerns about originality and
authenticity are growing. Some artists embrace AI as a collaborative tool, while
others worry about job displacement and creative devaluation.
"""

analysis = await kernel.invoke(
    analyze_info,
    content=sample_content,
    focus="impact on creative process"
)
print("\nAnalysis:\n", analysis)

# Example 3: Synthesize findings
synthesis = await kernel.invoke(
    synthesize_findings,
    elements=f"{str(breakdown)}\n\n{str(analysis)}",
    audience="technology policy makers"
)
print("\nSynthesis:\n", synthesis)

# Store research in memory
await memory.save_information(
    collection="research_findings",
    id="ai_creative_impact",
    text=f"Topic Breakdown:\n{breakdown}\n\nAnalysis:\n{analysis}\n\nSynthesis:\n{synthesis}"
)

### 4. Search Research Findings

Try searching through your stored research!

In [ ]:
# Search for research elements
search_terms = ["creative process", "economic impact", "future implications"]

for term in search_terms:
    print(f"\nSearching findings related to '{term}':\n")
    results = await memory.search("research_findings", term)
    for result in results:
        print(f"Match (relevance: {result.relevance}):\n{result.text}\n")

### 5. Additional exercises

Try these exercises to experiment with research prompts:

1. Modify the breakdown_prompt to:
   - Add stakeholder analysis
   - Include historical context
   - Map interdisciplinary connections

2. Enhance the analysis_prompt with:
   - Comparative frameworks
   - Risk assessment
   - Future scenarios

3. Expand the synthesis_prompt to include:
   - Different audience adaptations
   - Visual summary suggestions
   - Action recommendations

4. Create a new prompt that:
   - Generates research questions
   - Designs investigation methods
   - Evaluates source credibility

5. Advanced Challenge:
   - Create a prompt that combines multiple research methods
   - Design a framework for comparing conflicting findings
   - Develop a system for tracking research evolution

Remember: Focus on how different prompt elements affect the research process. Experiment with various analytical approaches!